# The GPS Sensor and Autonomous Vehicles 

<div style="background-color:black;color:white;padding:20px;">
<p>M. I. Pedro José Correa Caicedo</p>
</div>

## 1) Sensors in autonomous vehicles: 

<img src="./images/sensores_car.png" alt="sensor" width="600" height="500"/>

##### Fig. 1. Sensors in autonomous vehicles.

## 2) Sub systems in autonomous vehicles: 

<img src="./images/sub sistemas.png" alt="sistem" width="700" height="500"/>

##### Fig. 2. Sub systems in autonomous vehicles.

## 3) GPS operation: 

<img src="./images/satelite.png" alt="satelite" width="900" height="400"/>

##### Fig. 3. GPS satellites. a) Triangulation. b) Obstacles to the signal. 

#### a) Satellite networks : Global Navigation Satellite Systems (GNSS) 

<p>1) Global Positioning System (GPS) </p>
<p>2) BeiDou Navigation Satellite System (BDS) </p>
<p>3) Galileo (European Union) </p>
<p>4) Globalnaya Navigazionnaya Sputnikovaya Sistema (GLONASS) </p>
<p>5) Indian Regional Navigation Satellite System (IRNSS) </p>
<p>6) Quasi-Zenith Satellite System (QZSS) </p>


|Red     | País              | Num. Satélites |
|--------|-------------------|----------------|
|GPS     | United States     |       30+      |
|BDS     | China             |       35       |
|Galileo | European Union    |       24+      |
|GLONASS | Russian Federation|       24+      |
|IRNSS   | India             |       7        |
|QZSS    | Japan             |       4        |

Ref: https://www.gps.gov/systems/gnss/

<img src="./images/meridi.PNG" alt="lalo" width="500" height="400"/>

Ref: https://journeynorth.org/tm/LongitudeIntro.html

#### b)Standard NMEA (National Marine Electronics Association)


<img src="./images/ST_GP.png" alt="satelite" width="900" height="400"/>

##### Fig. 4. Standard NMEA a) GPGGA. b) GPRMC.  

|NMEA Header       | Value                  | Description
|------------------|------------------------|--------------------------|
| StartChar        | $                      | ASCII for 36             |
| SentenceName     | GPGGA                  | Identifier               |
| Separator        | ,                      | ASCII for 44             |
| Data_1 to Data_N | 083445.00,1256.60109,N | latitude and longitude   |
| Fix Quality      | 1 or *                 | Quality, parameter       |
| HDOP             | 1 or *                 | Horizontal dilution precision|
| CheckSum         | For example, 7E        | HEX 8-bit exclusive OR       |

Ref: 
https://es.mathworks.com/help/supportpkg/arduino/ref/read-serial-data-from-a-gps-shield-using-arduino-hardware.html


<p>Latitude and longitude are given in the format  DDMM.MM:
    
DD = Degrees 

MM.MM = Minutes  + (Seconds / 60)</p>

## 4) GPS data acquisition:

#### Pmod GPS Module Connection (Digilent)

<img src="./images/conec.png" alt="satelite" width="900" height="400"/>



##### Fig. 5. Sensor connection diagrams. a) connection to the PC, b) connection to the Raspberry Pi. 



Driver ttl to usb: https://rogerbit.com/wprb/2021/04/solucion-a-pl2303hxa-phased-out-since-2012-please-contact-your-supplier-en-windows-10/

#### a)Import library: 

In [1]:
import serial
import time
import csv as txt
import pynmea2
from datetime import datetime
import matplotlib.pyplot as plt

#### b) Start serial port: 

In [2]:
port="COM4"
br=9600
ser=serial.Serial(port,br, timeout=0.5)

#### c) Function to read the serial port: 

In [3]:
def gpsread():
    gpsread=ser.readline()
    return gpsread

<img src="./images/datos puros.PNG" alt="satelite" width="700" height="400"/>


##### Fig. 6. Raw Data. 

#### d) Save the raw data: 

In [4]:
def gps_raw():
    gps=gpsread()
    gps_s=gps.decode('utf-8')
    savegps=open('Raw_data.txt','a')
    savegps.write(gps_s)
    
    return gps_s

<img src="./images/datos lim.PNG" alt="satelite" width="600" height="400"/>

##### Fig. 7. Clean data.  

#### e) Function to separate the data of interest: 

In [5]:
def latlon():
    gps_s=gps_raw()
    global LAT,LON,LAT_D,LON_D,NUMSAT,TIM
    LAT='Null'
    LON='Null'
    NUMSAT='Null'
    TIM='Null'
    LAT_D='Null'
    LON_D='Null'
    #if gps.find(b'GGA') > 0:
    if gps_s[0:6]=='$GPGGA':
        msg = pynmea2.parse(gps_s)
        LAT=(msg.latitude)
        LAT_D=(msg.lat_dir)
        LON=(msg.longitude)
        LON_D=(msg.lon_dir)
        TIM=(msg.timestamp) 
        NUMSAT=(msg.num_sats)
        #print(LAT_D,LON_D)
            
    return LAT,LAT_D,LON,LON_D,TIM,NUMSAT
    

<p>Lat=201836.000; Lon=10049.3330
    
  Data= DDMM.MM

.d = MM.MM / 60
    
Degrees with decimals  = DD + .d 

Latitude= 20.54221666666667; Longitude=-100.82304166666667</p>

#### f) Run the program and save the data in a .csv: 

In [6]:
save=open('gps_adq.csv','w')
save.write("Lat,Lat_d,Lon,Lon_d,timgps,numsat\n")

if __name__ == '__main__':
    try:
        while True:
            LAT,LAT_D,LON,LON_D,TIM,NUMSAT=latlon()
            if LAT !='Null':
                print(LAT,LAT_D,LON,LON_D,TIM,NUMSAT)
                save=open('gps_adq.csv','a')      
                save.write(str(LAT))
                save.write(',')
                save.write(str(LAT_D))
                save.write(',')
                save.write(str(LON))
                save.write(',')
                save.write(str(LON_D))
                save.write(',')
                save.write(str(TIM))
                save.write(',')
                save.write(str(NUMSAT))
                save.write('\n')

            # Reset:CTRL + C
    except KeyboardInterrupt:
        print("Stop")

20.54241 N -100.8228 W 22:02:21 6
20.542361666666668 N -100.822805 W 22:02:22 6
20.542328333333334 N -100.82283166666667 W 22:02:23 7
20.54237 N -100.82283333333334 W 22:02:24 7
20.54237 N -100.822825 W 22:02:25 7
20.542386666666665 N -100.822825 W 22:02:26 7
20.54240666666667 N -100.822835 W 22:02:27 8
20.542395 N -100.82285166666666 W 22:02:28 8
20.542403333333333 N -100.82285666666667 W 22:02:29 8
20.542401666666667 N -100.82285 W 22:02:30 8
20.542368333333332 N -100.82283 W 22:02:31 8
20.542331666666666 N -100.822845 W 22:02:32 8
20.542346666666667 N -100.82284666666666 W 22:02:33 8
20.542361666666668 N -100.82283666666666 W 22:02:34 8
20.54238 N -100.82282166666667 W 22:02:35 8
20.542381666666667 N -100.82281333333333 W 22:02:36 8
20.542383333333333 N -100.82280666666666 W 22:02:37 8
Stop
